# Data processing: step 2

Combine the fj output with the muons that were previously set aside

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
code = "od"
#code = "cmssim"
#code = "wp_wzpythia_forcms_charge-mz50.0-mw40.0-mwp1000_full"
#code = "wp_wzpythia_forcms_charge-mz70.0-mw60.0-mwp1200_full"

start_read, stop_read, chunk_size = 0, 1980000, 10000

update_freq = 10*chunk_size

current_chunk_start = start_read

n_features = 13 # 13 w/ iso variables, 9 w/o

In [3]:
full_data_array = np.empty((1, n_features))
# dimu pi eta phi M jet pi eta phi M tau21


In [4]:
while current_chunk_start < stop_read:
    
    loc_data_array = np.empty((1, n_features))
    
    start_ind, stop_ind = current_chunk_start, current_chunk_start+chunk_size
    
    if start_ind % update_freq == 0:
        print(f"Analyzing events {start_ind} to {stop_ind}...")
    
    infile_dimuons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_only_{start_ind}_{stop_ind}_{code}.dat"
    infile_muiso = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_iso_{start_ind}_{stop_ind}_{code}.dat"
    infile_jets = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/jet_obs_{start_ind}_{stop_ind}_{code}.dat"

    
    ifile_muons = open(infile_dimuons, "r")
    ifile_iso = open(infile_muiso, "r")
    ifile_jets = open(infile_jets, "r")
    
    read_muons = ifile_muons.readlines()
    read_iso = ifile_iso.readlines()
    read_jets = ifile_jets.readlines()
    

    if len(read_muons) != len(read_jets):
        print("ERROR! Different number of muon and jet events.")
        print(len(read_muons), len(read_jets))
        break
        
        
    for i in range(len(read_muons)):
        if "#" in read_muons[i]: # at begin or end event
            pass
        else:
            loc_event = []

            # jets
            jet_values = read_jets[i].split()
            loc_event.append(float(jet_values[0])) # jet pt
            loc_event.append(float(jet_values[1])) # jet eta
            loc_event.append(float(jet_values[2])) # jet phi
            loc_event.append(float(jet_values[3])) # jet M
            loc_event.append(float(jet_values[4])) # jet tau21
            
            # muon isolation
            iso_values = read_iso[i].split()
            loc_event.append(float(iso_values[0])) # mu R = 0.3
            loc_event.append(float(iso_values[1])) # mu R = 0.5
            loc_event.append(float(iso_values[2])) # amu R = 0.3
            loc_event.append(float(iso_values[3])) # amu R = 0.5

            # muons
            dimu_values = read_muons[i].split()
            loc_event.append(float(dimu_values[0])) # dimuon pt
            loc_event.append(float(dimu_values[1])) # dimuon eta
            loc_event.append(float(dimu_values[2])) # dimuon phi
            loc_event.append(float(dimu_values[3])) # dimuon M
            

            loc_event = np.array(loc_event).reshape(1, n_features)

            loc_data_array = np.vstack((loc_data_array, loc_event))
        
    
    ifile_muons.close()
    ifile_iso.close()
    ifile_jets.close()
    
    full_data_array = np.vstack((full_data_array, loc_data_array[1:] ))
    current_chunk_start += chunk_size

Analyzing events 0 to 10000...
Analyzing events 100000 to 110000...
Analyzing events 200000 to 210000...
Analyzing events 300000 to 310000...
Analyzing events 400000 to 410000...
Analyzing events 500000 to 510000...
Analyzing events 600000 to 610000...
Analyzing events 700000 to 710000...
Analyzing events 800000 to 810000...
Analyzing events 900000 to 910000...
Analyzing events 1000000 to 1010000...
Analyzing events 1100000 to 1110000...
Analyzing events 1200000 to 1210000...
Analyzing events 1300000 to 1310000...
Analyzing events 1400000 to 1410000...
Analyzing events 1500000 to 1510000...
Analyzing events 1600000 to 1610000...
Analyzing events 1700000 to 1710000...
Analyzing events 1800000 to 1810000...
Analyzing events 1900000 to 1910000...


In [5]:
full_data_array = full_data_array[1:] 
print("Done!")
print(f"Data array has shape {full_data_array.shape}")

Done!
Data array has shape (1957364, 13)


In [6]:


np.save(f"compiled_data/{code}", full_data_array)